# Setup and Imports
This cell sets up the environment and imports necessary libraries for the project.

* Installs required packages (accelerate, transformers, datasets, peft)
* Imports necessary Python libraries (os, torch, datetime, datasets, warnings)
* Imports specific modules from transformers and peft
* Suppresses warnings to keep the output clean.


---

# Introduction to Parameter Efficient Fine-Tuning:

As large language models (LLMs) like GPT-3.5, LLaMA2, and PaLM2 increase in size, fine-tuning them for specific NLP tasks becomes more resource-intensive.

Parameter-Efficient Fine-Tuning (PEFT) helps reduce computational and memory demands by only adjusting a small set of parameters while keeping most of the model frozen. This prevents losing pre-learned information and allows fine-tuning with minimal compute.

The modular design of PEFT allows the same pretrained model to be used for multiple tasks by adding small, task-specific parameters, avoiding the need for storing full model copies.

The PEFT library simplifies the process by integrating techniques such as LoRA, Prefix Tuning, AdaLoRA, and more with popular tools like Transformers and Accelerate, enabling scalable fine-tuning of large models.

In [1]:
! pip install --upgrade transformers
! pip install -q accelerate pyboxen datasets==2.17.0 peft==0.4.0 pyboxen
! pip install --upgrade peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.10.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.4.0
    Uninstalling peft-0.4.0:
      Successfully uninstalled peft-0.4.0


In [2]:
import os
import torch
from datetime import datetime
from datasets import load_dataset
import warnings
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import prepare_model_for_kbit_training, PeftModel, LoraConfig, get_peft_model

warnings.filterwarnings('ignore')
import requests

# Download Dataset

In [3]:
# URLs for the raw files
test_url = "https://raw.githubusercontent.com/initmahesh/MLAI-community-labs/main/Class-Labs/Lab-3(Fine-tuning-PEFT-LoRA)/formatted_test_set.jsonl"
train_url = "https://raw.githubusercontent.com/initmahesh/MLAI-community-labs/main/Class-Labs/Lab-3(Fine-tuning-PEFT-LoRA)/formatted_train_set.jsonl"

# Download the files
test_file_path = "/content/formatted_test_set.jsonl"
train_file_path = "/content/formatted_train_set.jsonl"

# Download and save the files
response_test = requests.get(test_url)
with open(test_file_path, "wb") as f_test:
    f_test.write(response_test.content)

response_train = requests.get(train_url)
with open(train_file_path, "wb") as f_train:
    f_train.write(response_train.content)

In [4]:
def prepare_training_data(train_file_path, test_file_path):
    """
    Prepare training data specifically for MSA analysis with page_number and content format
    """
    import json

    with open(train_file_path, 'r') as f:
        train_data = [json.loads(line) for line in f]

    formatted_train_data = []
    for item in train_data:
        # Extract the page content and combine if needed
        contents = []
        if isinstance(item.get('content'), list):
            for page in item['content']:
                if isinstance(page, dict) and 'content' in page:
                    contents.append(f"Page {page.get('page_number', '')}: {page['content']}")
        else:
            contents.append(item.get('content', ''))

        # Create a structured prompt
        prompt = f"""
Context: Master Service Agreement content:
{' '.join(contents)}

Question: {item.get('question', '')}

Answer: {item.get('answer', '')}
"""
        formatted_train_data.append({
            "text": prompt.strip()
        })

    # Save formatted training data
    with open('formatted_train.jsonl', 'w') as f:
        for item in formatted_train_data:
            f.write(json.dumps(item) + '\n')

    return 'formatted_train.jsonl'

# Call the function
formatted_train_path = prepare_training_data(train_file_path, test_file_path)

# Load Model and Tokenizer
This cell loads the pre-trained model and tokenizer.

* Specifies the model name: "microsoft/phi-1_5" (a lightweight model)
* Loads the pre-trained causal language model
* Loads the corresponding tokenizer
* Sets the pad token to be the same as the end-of-sequence token


In [6]:
# Load the model and tokenizer
model_name = "microsoft/phi-1_5"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

# Evaluate Out-of-the-Box Model Performance
This cell sets up and runs inference using the original, untrained model:

* Defines a function to generate responses from the model
* Loads questions from the test set
* Extracts relevant parts related to "Governing Law"
* Constructs a focused prompt for the model
* Runs inference with the out-of-the-box model and displays the result

In [7]:
from pyboxen import boxen

def generate_response(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)

    # Move inputs to the same device as the model
    device = next(model.parameters()).device  # Get the model's device
    inputs = inputs.to(device)  # Move inputs to the model's device

    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=100,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Load test question
import json
with open(test_file_path, "r") as f:
    questions = [json.loads(line) for line in f]

question = "Does this contract has a provision that mentions that customer data will be deleted upon request by customer or after closure or termination of agreement for whatsoever reason?"

relevant_parts = [
    part['content'] for part in questions
    if 'Governing Law' in part['content'] or 'governed by' in part['content'].lower()
]

prompt = f"""
As a legal expert, analyze the following excerpts from a Master Service Agreement:

{' '.join(relevant_parts)}

{question}

Provide a concise answer.
"""

print("\n" + "=" * 50)
print("Results Before Fine-Tuning".center(50))
print("=" * 50 + "\n")

response = generate_response(prompt, model, tokenizer)
print(boxen(
    response,
    title="Out-of-the-box model response",
    padding=1,
    margin=1,
    color="blue"
))


            Results Before Fine-Tuning            



This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


                                                                                                                   
   ╭─ Out-of-the-box model response ───────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │                                                                                                           │   
   │   As a legal expert, analyze the following excerpts from a Master Service Agreement:                      │   
   │                                                                                                           │   
   │   You are a seasoned lawyer with a strong background in Master Service Agreement agreement.\              │   
   │       Your expertise is required to analyze a Master Service Agreement agreement and answer a question    │   
   │   based on that Master Service Agreement agreement.                

## What is LoRA


Large Language Models (LLMs) are powerful tools for processing and understanding language, but fine-tuning them for specific tasks can be challenging because of their enormous size and computational demands. This is where Low-Rank Adaptation (LoRA) comes in, offering an efficient solution for fine-tuning LLMs without needing to adjust every parameter.

Instead of modifying the entire model, LoRA focuses on a small, manageable subset of parameters. Here’s a simplified breakdown of how it works:

1. Normally, LLMs use a large matrix of parameters (W0) to make decisions. This matrix is huge and computationally expensive to adjust.

2. LoRA introduces two smaller matrices, A and B, which are much narrower than W0. These matrices represent a low-rank update to the model.

3. Instead of retraining the entire matrix W0, LoRA modifies only these smaller matrices, making the fine-tuning process much faster and more efficient. The result is a model update that’s nearly as effective as full fine-tuning but requires significantly fewer computational resources.

4. In a typical LLM layer, the output is calculated as output = W0x + b0. LoRA adds a new term, BAx, where A and B are the smaller matrices. This allows the model to adapt to new tasks without modifying the original large matrix W0.

![Image Description 2](https://drive.google.com/uc?export=view&id=1XnPMJzKwHun6SGkoUgxDtAzozcIxRRTA)


*Source: [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)*


# Hyper-Parameters for LoraConfig

## r (Rank):

This defines the rank of the low-rank decomposition matrices A and B. A higher rank means more parameters to fine-tune and potentially better performance, but at the cost of increased memory and compute.
Value: 16 is moderate rank value balancing efficiency and expressiveness.

##lora_alpha:

This is a scaling factor applied to the updates from the low-rank matrices A and B before adding them to the original weight matrix W0. It controls how much influence the LoRA layers have over the original model.
Value: 32, gives moderate influence to the LoRA updates.

## target_modules:

These are the specific layers in the model where LoRA is applied. Only these layers will be fine-tuned with LoRA. Examples here include:
1. "o_proj": The output projection layer.
2. "qkv_proj": The query, key, and value projections in the transformer.
3. "gate_up_proj", "up_proj", "down_proj", "lm_head".

## bias:

Determines whether LoRA will also adjust the bias terms in the model. In this case, "none" indicates that the bias terms are not fine-tuned, meaning only weights are updated.


## lora_dropout:

The dropout rate applied to LoRA layers during training. Dropout helps regularize the model by randomly ignoring some updates during training, reducing overfitting.
Value: 0.05 (5% dropout), meaning that 5% of the connections are dropped during fine-tuning.

## task_type:

The task type that the model is being fine-tuned for. In this case, "CAUSAL_LM" means the task is Causal Language Modeling, where the model is predicting the next word or token in a sequence.



In [8]:
# 1. First prepare the model
print("Preparing model for LoRA fine-tuning...")
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

Preparing model for LoRA fine-tuning...


In [9]:
# 2. Configure LoRA
print("Configuring LoRA...")
lora_config = LoraConfig(
    r=32,                # LoRA attention dimension
    lora_alpha=64,       # Alpha scaling
    target_modules=[
        "o_proj",
        "qkv_proj",
        "gate_up_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.1,    # Dropout probability
    task_type="CAUSAL_LM"
)

Configuring LoRA...


In [10]:
# 3. Get PEFT model
print("Applying LoRA to model...")
model = get_peft_model(model, lora_config)

Applying LoRA to model...


In [11]:
# 4. Print trainable parameters info
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,d} || "
        f"all params: {all_param:,d} || "
        f"trainable%: {100 * trainable_params / all_param:.2f}%"
    )

print_trainable_parameters(model)

trainable params: 1,703,936 || all params: 1,419,974,656 || trainable%: 0.12%


# Transforms MSA (Master Service Agreement) data into tokenized format for model processing
Input Handling:

Accepts either direct text or question-answer pairs
Adds legal expert context to prompt


Key Operations:

* Tokenizes text with 512 token limit
* Applies maximum length padding
* Creates attention mask (all 1s)
* Generates matching labels

In [12]:
def generate_and_tokenize_prompt(example):
    """
    Modified tokenization for MSA-specific format
    Handles both train and validation dataset schemas
    """
    # Format the prompt to include system context for legal analysis
    system_context = "You are a legal expert analyzing a Master Service Agreement."

    # Access 'text' if available, otherwise construct from 'question' and 'answer'
    text = example.get('text', '')  # Get 'text' if present, otherwise empty string
    if not text:  # If 'text' is empty
        text = f"Question: {example.get('question', '')}\nAnswer: {example.get('answer', '')}"

    prompt = f"{system_context}\n\n{text}{tokenizer.eos_token}"

    encoded = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )

    encoded["attention_mask"] = [1] * len(encoded["input_ids"])
    encoded["labels"] = encoded["input_ids"].copy()

    return encoded

In [13]:
# Load datasets
train_dataset = load_dataset('json', data_files=formatted_train_path, split='train')
validation_dataset = load_dataset('json', data_files=test_file_path, split='train')

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(
    generate_and_tokenize_prompt,
    remove_columns=train_dataset.column_names
)
tokenized_validation_dataset = validation_dataset.map(
    generate_and_tokenize_prompt,
    remove_columns=validation_dataset.column_names
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

## Set Up Trainer and Start Training

This cell sets up the training configuration and starts the fine-tuning process.

- Sets up TrainingArguments with various hyperparameters
- Creates a Trainer instance with the model, datasets, and training arguments
- Starts the training process

![Image Description](https://drive.google.com/uc?export=view&id=1evbDx1GhJy907b1BEs5SbMcLSXl7hiYE)

*Source: [Guide to Fine-Tuning LLMs with LoRA and QLoRA](https://www.mercity.ai/blog-post/guide-to-fine-tuning-llms-with-lora-and-qlora)*


In [14]:
# Set up the trainer
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Disable wandb integration by setting report_to to "none"
output_dir = "./phi-1_5-finetune-msa"
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    args=TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=1,
        warmup_ratio=0.1,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        max_steps=100,
        learning_rate=5e-4,
        fp16=True,
        logging_dir="./logs",
        logging_steps=10,
        save_strategy="steps",
        save_steps=50,
        evaluation_strategy="steps",
        eval_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
        report_to="none"  # Disable wandb integration
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# Train the model
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
50,1.894100,1.330742
100,1.662800,0.877684


TrainOutput(global_step=100, training_loss=1.9665080642700195, metrics={'train_runtime': 481.7454, 'train_samples_per_second': 3.321, 'train_steps_per_second': 0.208, 'total_flos': 5151050484940800.0, 'train_loss': 1.9665080642700195, 'epoch': 25.0})

In [15]:
# 8. Save the LoRA model
print("Saving LoRA model...")

# Merge and unload the LoRA weights before saving
model.merge_and_unload()

# Save the merged model with adapter weights
# Create the directory if it doesn't exist
import os
os.makedirs("./phi-1_5-lora-msa-final", exist_ok=True)
model.save_pretrained("./phi-1_5-lora-msa-final")

# Manually save the adapter weights
torch.save(model.state_dict(), "./phi-1_5-lora-msa-final/adapter_model.bin") # Save adapter weights to adapter_model.bin

Saving LoRA model...


In [16]:
# 9. For inference, load the LoRA model properly:
def load_lora_model(base_model, lora_path):
    # Load the LoRA config and model
    config = LoraConfig.from_pretrained(lora_path)
    lora_model = get_peft_model(base_model, config)
    # Load adapter weights from adapter_model.bin, but only LoRA-related weights
    # Load weights to CPU first to avoid CUDA OOM
    adapter_weights = torch.load(f"{lora_path}/adapter_model.bin", map_location=torch.device('cpu'))
    lora_model.load_state_dict(adapter_weights, strict=False) # strict=False ignores missing keys
    # Move the model to GPU if available
    if torch.cuda.is_available():
        lora_model.to(torch.device('cuda'))
    return lora_model

# Compare Out-of-the-Box and Fine-Tuned Model Responses
This cell compares the performance of the original and fine-tuned models:

* Reloads the test questions and constructs the prompt (for consistency)
* Defines a function to generate responses from any given model
* Runs inference with the original out-of-the-box model
* Runs inference with the fine-tuned model
* Displays both responses for direct comparison
* Allows for analysis of improvements in accuracy, relevance, and quality

In [17]:
from transformers import pipeline
from pyboxen import boxen
import gc
import torch

def generate_improved_response(prompt, model, tokenizer):
    """
    Generates an improved response using the provided model and tokenizer.

    Args:
        prompt (str): The input prompt.
        model: The language model to use for generation.
        tokenizer: The tokenizer associated with the model.

    Returns:
        str: The generated response.
    """
    pipe = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )
    response = pipe(prompt, max_new_tokens=100)[0]['generated_text']
    return response.split(prompt)[-1].strip()  # Extract only the generated part

# Test prompt
test_prompt = """
Question: Does this contract has a provision that mentions that customer data will be deleted upon request by customer or after closure or termination of agreement for whatsoever reason?
Context: [Your MSA content here]
"""

def print_header(text):
    print("\n" + "=" * 60)
    print(text.center(60))
    print("=" * 60 + "\n")

# Test base model
print_header("Testing Base Model")

# Clear cache before inference
gc.collect()
torch.cuda.empty_cache()

with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
    base_response = generate_improved_response(test_prompt, model, tokenizer)

print(boxen(
    base_response,
    title="Base Model Response",
    padding=1,
    margin=1,
    color="blue"
))

# Test LoRA model
print_header("Testing LoRA Model")

# Clear cache before inference for LoRA model
gc.collect()
torch.cuda.empty_cache()

with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
    lora_model = load_lora_model(model, "./phi-1_5-lora-msa-final")
    lora_response = generate_improved_response(test_prompt, lora_model, tokenizer)

print(boxen(
    lora_response,
    title="LoRA Model Response",
    padding=1,
    margin=1,
    color="green"
))


                     Testing Base Model                     



Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianFor

                                                                                                                   
   ╭─ Base Model Response ─────────────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Answer: No, this contract does not mention any provision regarding customer data deletion.              │   
   │                                                                                                           │   
   │   Question: Based on the information provided in this question, is it possible that the customer data     │   
   │   deletion provision mentioned in this contract is not enforceable?                                       │   
   │   Context: [Your MSA content here]                                                                        │   
   │                                                                    

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianFor

                                                                                                                   
   ╭─ LoRA Model Response ─────────────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Answer: No, this contract does not mention any provision regarding customer data deletion.              │   
   │                                                                                                           │   
   │   Question: Based on the information provided in this question, is it possible that the customer data     │   
   │   deletion provision mentioned in this contract is not enforceable?                                       │   
   │   Context: [Your MSA content here]                                                                        │   
   │                                                                    